In [1]:
# ======================================================================
# SCRIPT 2 — CLASSIFICATION SUPERVISÉE (SAheart)
# ======================================================================

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from patsy import dmatrix


In [2]:

# ======================================================================
# 1) LECTURE + PREP
# ======================================================================

don = pd.read_csv("SAheart.data", sep=",")
don = don.rename(columns={"chd": "Y"})
don = don.drop(columns=["row.names"])

Y = don["Y"].to_numpy()

# design matrix
nomsvar = list(don.columns.difference(["Y"]))
formule = "~" + "+".join(nomsvar)
dsX = dmatrix(formule, don)
X = np.asarray(dsX)[:, 1:]


In [3]:

# ======================================================================
# 2) BLOCS VC EXTERNE
# ======================================================================
nb = 10
tmp = np.arange(don.shape[0]) % nb
rng = np.random.default_rng(seed=1234)
bloc = rng.choice(tmp, size=don.shape[0], replace=False)

PROB = pd.DataFrame({
    "bloc": bloc,
    "Y": don["Y"],
    "log": 0.0,
    "lasso": 0.0,
    "ridge": 0.0,
    "elast": 0.0
})

# Grille pour pénalisation
def grille(X, y, type="lasso", ng=400):
    scalerX = StandardScaler().fit(X)
    Xcr = scalerX.transform(X)
    l0 = np.abs(Xcr.T.dot((y - y.mean()))).max() / X.shape[0]
    llc = np.linspace(0, -4, ng)
    if type == "lasso":
        Cs = 1 / 0.9 / X.shape[0] / (l0 * 10**llc)
    elif type == "ridge":
        Cs = 1 / 0.9 / X.shape[0] / ((l0 * 10**llc) * 100)
    elif type == "enet":
        Cs = 1 / 0.9 / X.shape[0] / ((l0 * 10**llc) * 2)
    return Cs


In [4]:

# ======================================================================
# 3) VC EXTERNE
# ======================================================================

for i in range(nb):
    print(f"\n### Bloc {i} ###")

    Xapp = X[bloc != i, :]
    Xtest = X[bloc == i, :]
    Yapp = don[bloc != i]["Y"]
    Ytest = don[bloc == i]["Y"]

    # -----------------------------------------
    # MODELE 1 : Logistique sans pénalisation
    # -----------------------------------------
    log = LogisticRegression(penalty=None, solver="newton-cholesky").fit(Xapp, Yapp)
    PROB.loc[PROB.bloc == i, "log"] = log.predict_proba(Xtest)[:, 1]

    # -----------------------------------------
    # MODELE 2 : LASSO
    # -----------------------------------------
    Cs_lasso = grille(Xapp, Yapp, "lasso")
    lassocv = LogisticRegressionCV(
        cv=10, penalty="l1", solver="saga", Cs=Cs_lasso, max_iter=2000
    )
    pipe_lasso = Pipeline([("cr", StandardScaler()), ("lassocv", lassocv)])
    pipe_lasso.fit(Xapp, Yapp)
    PROB.loc[PROB.bloc == i, "lasso"] = pipe_lasso.predict_proba(Xtest)[:, 1]

    # -----------------------------------------
    # MODELE 3 : RIDGE
    # -----------------------------------------
    Cs_ridge = grille(Xapp, Yapp, "ridge")
    ridgecv = LogisticRegressionCV(
        cv=10, penalty="l2", solver="saga", Cs=Cs_ridge, max_iter=2000
    )
    pipe_ridge = Pipeline([("cr", StandardScaler()), ("ridgecv", ridgecv)])
    pipe_ridge.fit(Xapp, Yapp)
    PROB.loc[PROB.bloc == i, "ridge"] = pipe_ridge.predict_proba(Xtest)[:, 1]

    # -----------------------------------------
    # MODELE 4 : ELASTIC NET
    # -----------------------------------------
    Cs_enet = grille(Xapp, Yapp, "enet")
    enetcv = LogisticRegressionCV(
        cv=10, penalty="elasticnet", solver="saga", l1_ratios=[0.5],
        Cs=Cs_enet, max_iter=2000
    )
    pipe_enet = Pipeline([("cr", StandardScaler()), ("enetcv", enetcv)])
    pipe_enet.fit(Xapp, Yapp)
    PROB.loc[PROB.bloc == i, "elast"] = pipe_enet.predict_proba(Xtest)[:, 1]



### Bloc 0 ###

### Bloc 1 ###

### Bloc 2 ###

### Bloc 3 ###

### Bloc 4 ###

### Bloc 5 ###

### Bloc 6 ###

### Bloc 7 ###

### Bloc 8 ###

### Bloc 9 ###


In [5]:

# ======================================================================
# 4) PERFORMANCE GLOBALE (AUC)
# ======================================================================

tab_auc = pd.DataFrame({
    "AUC_log":   [roc_auc_score(PROB["Y"], PROB["log"])],
    "AUC_lasso": [roc_auc_score(PROB["Y"], PROB["lasso"])],
    "AUC_ridge": [roc_auc_score(PROB["Y"], PROB["ridge"])],
    "AUC_elast": [roc_auc_score(PROB["Y"], PROB["elast"])]
})

print("\n=== AUC ===")
print(tab_auc)

# Sauvegarde
PROB.to_csv("PROB_classif.csv", index=False)
tab_auc.to_csv("perf_classif.csv", index=False)



=== AUC ===
    AUC_log  AUC_lasso  AUC_ridge  AUC_elast
0  0.771151   0.764342   0.773634   0.769785


# ARBRE DE DECISION 

In [6]:
from sklearn.tree import DecisionTreeClassifier
for i in np.arange(nb):
    print(i)
    Xapp = X[bloc!=i,:]
    Xtest = X[bloc==i,:]
    Yapp = don[bloc!=i]["Y"]
    Ytest = don[bloc==i]["Y"]
    #### arbre
    arbre = DecisionTreeClassifier(min_samples_leaf=5)
    arbre.fit(Xapp,Yapp)
    PROB.loc[PROB.bloc==i,"arbre"] = arbre.predict_proba(Xtest)[:,1]

0
1
2
3
4
5
6
7
8
9


# RANDOM FOREST

In [9]:
from sklearn.ensemble import RandomForestClassifier
for i in np.arange(nb):
    print(i)
    Xapp = X[bloc!=i,:]
    Xtest = X[bloc==i,:]
    Yapp = don[bloc!=i]["Y"]
    ##### Foret
    foret = RandomForestClassifier()
    foret.fit(Xapp,Yapp)
    PROB.loc[PROB.bloc==i,'foret100'] = foret.predict_proba(Xtest)[:,1]

0
1
2
3
4
5
6
7
8
9


# GRADIENT BOOST

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
params = dict(max_iter=1000, max_depth=5, learning_rate=0.1, random_state=42)
hgbm = HistGradientBoostingClassifier(**params)
for i in np.arange(nb):
    print(i)
    Xapp = X[bloc!=i,:]
    Xtest = X[bloc==i,:]
    Yapp = don[bloc!=i]["Y"]
    ##### Foret
    hgbm.fit(Xapp,Yapp)
    PROB.loc[PROB.bloc==i,'hgbm'] = hgbm.predict_proba(Xtest)[:,1]

# ADA BOOST

In [10]:
from sklearn.ensemble import AdaBoostClassifier
params = dict(n_estimators=1000, learning_rate=0.1, random_state=42)
ada = AdaBoostClassifier(**params)
for i in np.arange(nb):
    print(i)
    Xapp = X[bloc!=i,:]
    Xtest = X[bloc==i,:]
    Yapp = don[bloc!=i]["Y"]
    ##### Foret
    ada.fit(Xapp,Yapp)
    PROB.loc[PROB.bloc==i,'ada1'] = ada.predict_proba(Xtest)[:,1]

0
1
2
3
4
5
6
7
8
9


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
params = dict(n_estimators=1000, max_depth=5, learning_rate=0.1, random_state=42)
gbm = GradientBoostingClassifier(
    **params,
    validation_fraction=0.1,
    n_iter_no_change=10,
)
for i in np.arange(nb):
    print(i)
    Xapp = X[bloc!=i,:]
    Xtest = X[bloc==i,:]
    Yapp = don[bloc!=i]["Y"]
    ##### Foret
    gbm.fit(Xapp,Yapp)
    print(gbm.n_estimators_)
    PROB.loc[PROB.bloc==i,'xgb5'] = gbm.predict_proba(Xtest)[:,1]

0
20
1
29
2
18
3
23
4
21
5
15
6
17
7
16
8
14
9
21


In [15]:
import sklearn.metrics as sklm
noms = PROB.columns[1:]
matsB = pd.DataFrame({"seuil": pd.Series(0.0, index=noms)})
s = .5
for nom in noms:
    matsB.loc[nom,"seuil"] = s
    confmat = sklm.confusion_matrix(PROB.Y, PROB.loc[:,nom]>=s)
    matsB.loc[nom, "tn"] = confmat[0,0]
    matsB.loc[nom, "tp"] = confmat[1,1]
    matsB.loc[nom, "fn"] = confmat[1,0]
    matsB.loc[nom, "fp"] = confmat[0,1]
    matsB.loc[nom,"sensitivity"] = confmat[1,1]/(confmat[1,1]+confmat[1,0])
    matsB.loc[nom,"specificity"] = confmat[0,0]/(confmat[0,0]+confmat[0,1])
    matsB.loc[nom,"sen+spe"] = matsB.loc[nom,"sensitivity"]+matsB.loc[nom,"specificity"]
    matsB.loc[nom,"accuracy"] = sklm.accuracy_score(PROB.Y, PROB.loc[:,nom]>=s)
print(matsB.round(3))

          seuil     tn     tp    fn    fp  sensitivity  specificity  sen+spe  \
Y           0.5  302.0  160.0   0.0   0.0        1.000        1.000    2.000   
log         0.5  251.0   81.0  79.0  51.0        0.506        0.831    1.337   
lasso       0.5  264.0   73.0  87.0  38.0        0.456        0.874    1.330   
ridge       0.5  263.0   73.0  87.0  39.0        0.456        0.871    1.327   
elast       0.5  267.0   70.0  90.0  35.0        0.438        0.884    1.322   
arbre       0.5  214.0   82.0  78.0  88.0        0.512        0.709    1.221   
foret100    0.5  247.0   68.0  92.0  55.0        0.425        0.818    1.243   
ada1        0.5  262.0   77.0  83.0  40.0        0.481        0.868    1.349   
xgb5        0.5  245.0   67.0  93.0  57.0        0.419        0.811    1.230   
hgbm        0.5  229.0   63.0  97.0  73.0        0.394        0.758    1.152   

          accuracy  
Y            1.000  
log          0.719  
lasso        0.729  
ridge        0.727  
elast        0